In [2]:
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import interpolate


with open(snakemake.input[0], 'rb') as in_:
    fig = pickle.load(in_)

ax = plt.gca()

In [4]:
%%capture
df = pd.read_csv(snakemake.input[1])
plt.clf()
fig, ax = plt.subplots(1, 1, figsize=(10 ,13))
plt.sca(ax)

In [5]:
scale = 1
df = df[df.ant_nb == 32]

def draw_trajectory(df, ax):
    x = df.path_x / scale
    y = df.path_y / scale
    ax.plot(x, y, alpha=0.75)
    return ax

def df_interpolate(df, max_=5):
    def per_df(d):
        x = np.array(d['path_x'])
        y = np.array(d['path_y'])

        # remove two consecutive equal points for interpolation
        okay = np.where(np.abs(np.diff(x)) + np.abs(np.diff(y)) > 0)
        x = np.r_[x[okay], x[-1]]
        y = np.r_[y[okay], y[-1]]

        tck,u=interpolate.splprep([x,y],s=0.0)
        x_i,y_i= interpolate.splev(np.linspace(0,1,(len(x)*10)),tck)

        new_rows = d.iloc[-1].to_dict()
        new_rows.update({'path_x' : x_i, 'path_y': y_i})
        return pd.DataFrame(new_rows)
    dfs = []
    agents = set(df["ant_nb"])
    trials = set(df["trial_nb"])
    if len(trials) > max_:
        trials = [arr[0] for arr in np.array_split(list(trials), max_)]
    for agent in agents:
        for trial in trials:
            dfs.append(per_df(df[(df["ant_nb"]==agent) & (df["trial_nb"]==trial)]))
    return pd.concat(dfs, ignore_index=True)


In [6]:
for ant in pd.unique(df["ant_nb"]):
    ant_df = df[df.ant_nb == ant]

    for run in pd.unique(ant_df["trial_nb"]):

        traj = ant_df[ant_df.trial_nb == run].copy(deep=True)
        
        traj.path_x = traj['path_x'] - traj.path_x.min()
        traj.path_y = traj['path_y'] - traj.path_y.min()
        
        draw_trajectory(df_interpolate(traj), ax)

heatmap, xedges, yedges = np.histogram2d(df.path_x, df.path_y, bins=(16,51))
ax.imshow(heatmap.T, origin='lower')
#fig
fig.savefig(snakemake.output[0])